In [1]:
import os
import json
import fasttext
from tqdm import tqdm
import json
# from transformers import AutoTokenizer

In [2]:
model = fasttext.load_model("/data/hyxu/codes/lowMT_compute/model/lid.176.ftz")
model

In [3]:
model.predict("sadnasjhfn iab asina winawfh siefnwvjawoi saf qwhoiasoidfw9 f as jfwoias;lk ")

(('__label__nl',), array([0.38591853]))

In [4]:

data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"
def _read(p):
    with open(p) as f:
        data = f.readlines()
    return [s.strip("\n") for s in data]
nl_data = _read(os.path.join(data_path, "train.nl"))
zh_data = _read(os.path.join(data_path, "train.zh"))

In [5]:
lang1, lang2 = "nl", "zh"
data_socre = {lang1: [], lang2: []}
r = 0.4
nl_data_result =[]
for s in tqdm(nl_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang1}" and result[1] > r:
        nl_data_result.append((s, result[1].item()))
zh_data_result =[]
for s in tqdm(zh_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang2}" and result[1] > r:
        zh_data_result.append((s, result[1].item()))

100%|██████████| 8000000/8000000 [02:11<00:00, 60879.49it/s]


In [6]:
len(nl_data_result), len(zh_data_result)

(1961790, 6871402)

In [15]:
nl_data_result = [(d[0], d[1].item()) for d in nl_data_result]

In [16]:
with open(os.path.join(data_path, "clean.train.nl"), "w") as f:
    json.dump(nl_data_result, f, indent=4)
with open(os.path.join(data_path, "clean.train.zh"), "w") as f:
    json.dump(zh_data_result, f, indent=4)

## 写比赛结果

In [1]:
import os
import json
import torch
from tqdm import tqdm
from transformers import MBartForConditionalGeneration, AutoTokenizer
from datasets import load_from_disk, DatasetDict, Dataset
import  xml.dom.minidom
from eval import generate, evaluate_fn

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"

/home/hyxu/anaconda3/envs/work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
path = "/data/hyxu/lowMT_compute/data/public_data/test"
def _read_xml(p):
    dom = xml.dom.minidom.parse(p)
    root = dom.documentElement
    seg_list = root.getElementsByTagName("seg")
    data = [s.firstChild.data for s in seg_list]
    print(len(data))
    return [s.rstrip("\n") for s in data]
data = _read_xml(os.path.join(path, "test.nl-zh.nl.xml"))
data

1000


['Bij een bezoek aan de plaatselijke school waren ze geschokt door de chaotische scenes die vaak in de klas voorkwamen.',
 'Ik zal in de volgende wedstrijd zeker meer aandacht besteden aan het passeren en de offensieve verandering.',
 'Denemarken, Zweden en Finland zijn de eerste landen ter wereld die warmtepomptechnologie gebruiken om stedelijk afvalwater te behandelen.',
 'De actualiteit van deze twee processen moet worden gegarandeerd.',
 'Ik kan niet anders zeggen dan dat ik heel trots ben.',
 'Met onze ruim 400 winkels zitten we echt overal in Nederland.',
 'De Philips Airfryer is de enige Airfryer met Fat Removal-technologie',
 'Ook zegt het college dat men wil streven naar minder inhuur.',
 'De beelden worden vervolgens beoordeeld met kunstmatige intelligentie.',
 'Bij opsporingsberichtgeving dient het OM strikt de Aanwijzing Opsporingsberichtgeving na te leven.',
 'Vandaag nodigen we 66e afgestudeerden van Tsinghua Universiteit uit om het verhaal van hun graduatiedistributie te

In [2]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean/chpk-28378/model/"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [18]:
predictions = generate(model, tokenizer, "nl_XX", "zh_CN", data, 32, 3, 128)

100%|██████████| 32/32 [00:27<00:00,  1.18it/s]


In [19]:
dom = xml.dom.minidom.parse(os.path.join(path, "test.nl-zh.zh.xml"))
root = dom.documentElement
seg_list = root.getElementsByTagName("seg")
for s, p in zip(seg_list, predictions):
    s.firstChild.data = p
with open(os.path.join(path, "test.nl-zh.zh5.xml"), "w") as f:
    dom.writexml(f)

## 用tokenizer 清洗数据

In [6]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer

MBartTokenizerFast(name_or_path='/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model', vocab_size=250027, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN']}, clean_up_tokenization_spaces=True)

In [ ]:
with open(os.path.join(data_path, "clean.train.nl")) as f:
    nl_data_result = json.load(f)
with open(os.path.join(data_path, "clean.train.zh")) as f:
    zh_data_result = json.load(f)
len(nl_data_result), len(zh_data_result)

In [30]:
r = 15              # 拼接阈值
class CleanData(object):
    r =15
    clean_data = []
    noise_data = []
    def add(self, s):
        l = len(s)
        if l > r:
            self.clean_data.append(s)
        else:
            if len(self.noise_data) < 1:
                self.noise_data.append(s)
            else:
                new_string = self.noise_data.pop()
                new_string += s
                self.add(new_string)
nl_clean_data = CleanData()        
zh_clean_data = CleanData()        

In [31]:
tokenizer.tokenize("你是个大事必")

['▁你', '是个', '大事', '必']

In [32]:
from tqdm import  tqdm
for s in tqdm(nl_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    nl_clean_data.add(tokenized_text)

  1%|          | 14289/1961790 [00:22<08:24, 3858.20it/s]

In [ ]:
for s in tqdm(zh_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    zh_clean_data.add(tokenized_text)

In [ ]:
len(nl_clean_data)

## 测试验证机分数

In [2]:
model_path  = "/data/hyxu/lowMT_compute/model/ft-clean-zh-nl/chpk-30405/model"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()

In [6]:
data = load_from_disk("/data/hyxu/lowMT_compute/data/public_data/dev_set")

In [13]:
metrics = ["bleu",]
s, t= "nl_XX", "zh_CN"
outputs = evaluate_fn(model, tokenizer, s, t, {s:data[s], t:data[t]}, batch_size=16, num_beams=3, max_length=128,
                        metrics=metrics)
outputs[0]

100%|██████████| 63/63 [00:32<00:00,  1.93it/s]


{'bleu': 21.835933005889682}

In [10]:
outputs[1]

['打扰了 我要坐哪班火车去威海?',
 '编辑们要勇于表达自己的想法 不管他们看起来有多疯狂',
 '但有证据表明 并非每个人都会这么做',
 'Uranium是一种重型、不稳定的原子,它在核爆炸中与二到三核的微型原子相连。',
 '参观圣地亚哥神庙十分值得 它被尊为中国古代建筑杰出的范例',
 '能源危机不是方法危机,更何况不是资源危机。',
 '它利用这些信息销售产品',
 '你在思考什么是科学知识 以及如何得到它',
 '债券是一种投资行为',
 '辩护学与法律援助密不可分。',
 '我可以吃任何东西 但我觉得吃中国菜更好吃',
 '因此,法律制定者不应该违背自然法则',
 '完成硕士学位后 你可以专攻你擅长的领域',
 '消费者购买商品时 有权利得到法律上的保证',
 '申请重审需满足以下条件:',
 '太热了,我们什么时候唱?',
 '采访就被取消了 不知情的访谈取消了',
 '想法是新闻产品的灵魂',
 '汤姆是第一个到达终点线的人',
 '斯梅斯于2012年3月23日正式入选荷兰国家队',
 '你可以称它为真正的家居设计 以预算为准',
 '未来将有思维系统应用的增长预期。',
 '那些没有取得大专学历的人,有可能无法取得相应学历。',
 '红袜主帅戈尔三次致命击打对方门柱',
 '这样程序员就能轻而易举地改变程序',
 '一些中学教材和学习指南是免费的。',
 '我想买运动服,我该穿几号?',
 '亲爱的 你告诉我你昨天在餐馆吃太多了 所以你不想吃早餐 对吧?',
 '我们会把资产净额还到你户头 再过几天你就可以出院了',
 '现在金属流行在时尚圈非常盛行',
 '手机也可以是直播视频中销售商品的中介。',
 '这距离的世界纪录是荷兰人的(2.46,49)。',
 '长矛会在一次引爆中起飞 至少有30米长的引爆线',
 '我每天喝一杯咖啡 但并不多',
 '与上届世锦赛不同的是 范·加拉让球队重磅引入了前锋 维吉尔・范·狄克',
 '这份遗嘱是在两位见证人见证下所签署的',
 '记者要清楚地看到媒体的本质',
 '奥马哈必须赢才能赢得 冠军。',
 '科林・欧拉恩的"超级侠"系列是红玫瑰中的一颗璀璨之星',
 '日本人有很多海鲜 你喜欢吃鱼吗?',
 '女人跟男人一样有投票权和自立权利',
 '具体金额还在庭审中审定',
 '这些应用没有成功地改进

In [8]:
from sacrebleu import BLEU
bleu = BLEU()
bleu.corpus_score(outputs[1], [outputs[2]])

BLEU = 12.04 44.8/17.1/8.1/4.0 (BP = 0.961 ratio = 0.962 hyp_len = 13064 ref_len = 13579)

### 结果

epoch
1           11.12
2           12.04
            12.43
            13.37

## plm数据增强

In [3]:
import torch
from src.data_process import add_span_mask_noise
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MBartForConditionalGeneration
model_path = "/data/hyxu/lowMT_compute/model/pre-denoising"
lang1, lang2 = "nl_XX", "zh_CN"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model1 = MBartForConditionalGeneration.from_pretrained(model_path).cuda()

with open("/data/hyxu/lowMT_compute/data/public_data/train/pair/clean.train.nl-zh.zh") as f:
    zh_data = f.readlines()

In [3]:

text = "这是一个新的注意力机制，可以提高大模型的推理可扩展性。它的工作原理是将键和值投影在多个头之间共享，而不会大幅降低性能。"
encoded = tokenizer(text, return_tensors="pt")
# mask_idx = torch.randint(0, len(encoded["input_ids"][0]), (1,len(encoded["input_ids"][0]) // 4))
encoded["input_ids"] = torch.tensor(add_span_mask_noise(tokenizer.vocab_size, encoded["input_ids"][0],
                                                        noise_density=0.15, mean_noise_span_length=1)).reshape(1, -1)
encoded["attention_mask"] = encoded["input_ids"] != tokenizer.pad_token_id
# encoded["input_ids"][0, mask_idx[0]] = tokenizer.mask_token_id
# encoded["input_ids"], 
tokenizer.batch_decode(encoded["input_ids"])

['这是一个<mask>注意力机制,可以提高大模型的zh_CN理可扩展性。它的工作vi_VN是将键tr_TR值投影在多个头si_LK共享,而不会大幅降低性能。</s>ru_RU']

In [20]:
for k,v in encoded.items():
    encoded[k] = v.cuda()
outputs = model.generate(**encoded, num_beams=4, decoder_start_token_id=tokenizer.lang_code_to_id[lang2])
tokenizer.batch_decode(outputs)

/data/hyxu/Pruner/src/transformers/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['zh_CN 这是一个新的注意力机制,它可以提高大模型的推理可扩展性。它的工作原理是将键和值投影在多个头之间共享,而不会大幅缩小,从而大大提高大模型的性能。</s>']

In [3]:
from tqdm import tqdm
tokenizer.src_lang = lang2
def add_noise():
    inputs_ids, attention_mask = [], []
    for s in tqdm(zh_data):
        encoded = tokenizer(s, return_tensors="pt")
        encoded["input_ids"] = torch.tensor(add_span_mask_noise(tokenizer.vocab_size, encoded["input_ids"][0],
                                                                noise_density=0.15, mean_noise_span_length=1)).reshape(1, -1)
        mask = torch.ones_like(encoded["input_ids"])
        inputs_ids += encoded["input_ids"].tolist()
        attention_mask += mask.tolist()
    return {"input_ids": inputs_ids, "attention_mask": attention_mask}
data = add_noise()

100%|██████████| 129684/129684 [00:54<00:00, 2384.07it/s]


In [16]:
data["input_ids"][:10], data["attention_mask"][:10]

([['因为事情就是这么发生的</s><mask>'],
  ['"Cathleen. Whitaker证明 <mask>集zh_CN一身"</s>vi_VN'],
  ['飞行<mask>们可以控制408号航班了</s>zh_CN'],
  ['驱逐了<mask>吗 我们必须杀了他</s>zh_CN'],
  ['<mask>我直言 现实世界是我的专长</s>zh_CN'],
  ['就为了调整<mask>你的职称?</s>zh_CN'],
  ['警察不得跟随 <mask>安全抵达边境</s>zh_CN'],
  ['打开电视看CNN</s><mask>'],
  ['我要把你带<mask>一个没有这些烦恼的地方</s>zh_CN'],
  ['你们两个跟我来</s><mask>']],
 [])

In [4]:
batch_size = 16
num_batch = len(data["input_ids"]) // batch_size if len(data["input_ids"])%batch_size==0 else len(data["input_ids"]) // batch_size + 1
batch_data = []
for i in tqdm(range(num_batch)):
    x = tokenizer.pad({"input_ids": data["input_ids"][i*batch_size: (i+1)*batch_size],
                       "attention_mask": data["attention_mask"][i*batch_size: (i+1)*batch_size]}, 
                      padding=True, max_length=128, return_tensors="pt")
    batch_data.append(x)
len(batch_data)

  0%|          | 0/8106 [00:00<?, ?it/s]You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/data/hyxu/Pruner/src/transformers/src/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
100%|██████████| 8106/8106 [00:01<00:00, 4360.20it/s]


8106

In [5]:
predictions = []
for x in tqdm(batch_data):
    for k, v in x.items():
        x[k] = v.cuda()
    outputs = model.generate(**x, num_beams=4, decoder_start_token_id=tokenizer.lang_code_to_id[lang2])
    predictions += tokenizer.batch_decode(outputs)
len(predictions), predictions[:10]

  0%|          | 0/8106 [00:00<?, ?it/s]/data/hyxu/Pruner/src/transformers/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 23/8106 [13:07<76:53:45, 34.25s/it]


KeyboardInterrupt: 

In [14]:
x["input_ids"].shape, x["attention_mask"].shape, outputs.shape, tokenizer.batch_decode(data["input_ids"][:10])

(torch.Size([16, 20]),
 torch.Size([16, 20]),
 torch.Size([16, 1024]),
 ['因为事情就是这么发生的</s><mask>',
  '"Cathleen<mask> Whitaker证明 可以集于一身zh_CN</s>vi_VN',
  '飞行员们可以控制<mask>号航班了</s>zh_CN',
  '<mask>逐了之后吗 我们必须杀了他</s>zh_CN',
  '恕我直言 现实世界是我的<mask>长</s>zh_CN',
  '就为了调整下<mask>职称?</s>zh_CN',
  '警察不得跟随<mask>我要安全抵达边境</s>zh_CN',
  '打开电视看CNN</s><mask>',
  '我要把你带到一个没有这些<mask>的地方</s>zh_CN',
  '你们两个跟我来</s><mask>'])

In [6]:
predictions

['zh_CN                                                                                                                                                           因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为因为

In [17]:
data = load_from_disk("/data/hyxu/lowMT_compute/data/public_data/train/pair/zh_CN-denosing")
len(data)

7337384

## BT

In [2]:
model_path  = "/data/hyxu/lowMT_compute/model/ft-clean-denoising/chpk-24324/model/"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()

In [20]:
mono_path = "/data/hyxu/lowMT_compute/data/public_data/train/mono/bt.nl"
src_lang, tgt_lang = "nl_XX", "zh_CN"
with open(mono_path) as f:
    nl_data = f.readlines()
nl_data = [s.strip("\n") for s in nl_data]
len(nl_data)

500000

In [25]:
src_data = nl_data
num_beams, batch_size = 3, 32
num_batch = len(src_data) // batch_size if len(src_data) % batch_size == 0 else len(src_data) // batch_size +1
predictions = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(num_batch)):
        x = tokenizer(src_data[i*batch_size:(i+1)*batch_size], padding=True, max_length=128, return_tensors="pt",truncation=True)
        for k, v in x.items():
            x[k] = v.to(model.device)
        l = len(x["input_ids"][0])
        
        outputs = model.generate(input_ids=x["input_ids"],
                         attention_mask=x["attention_mask"],
                         num_beams=num_beams,
                         decoder_start_token_id=tokenizer.lang_code_to_id[tgt_lang],
                         max_length= int(l*1.5),)            #mbart
        predictions += tokenizer.batch_decode(outputs.tolist(), skip_special_tokens=True)

  1%|          | 159/15625 [06:00<9:18:30,  2.17s/it] 

In [1]:
predictions[:100]

NameError: name 'predictions' is not defined

In [4]:
# model
predictions = generate(model, tokenizer, src_lang, tgt_lang, nl_data, 32, 4, 128)
len(predictions)

  0%|          | 0/15625 [00:00<?, ?it/s]/data/hyxu/Pruner/src/transformers/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 26/15625 [04:24<44:02:57, 10.17s/it] 


KeyboardInterrupt: 